In [1]:
from IPython.display import Image
from pathlib import Path
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
import math
import xgboost as xgb

In [2]:
train_labels = pd.read_csv("KD/train_labels_KD.csv", index_col="seq_id")

In [3]:
c = ['seq_id'] + ['c_' + str(i) for i in list(range(1000))] + ['c2_' + str(i) for i in list(range(1000))] + ['c3_' + str(i) for i in list(range(1000))] +['x']
good_classes = [9,101, 103, 104, 138, 149,269, 271, 272,274,275, 276, 277, 280 ,288,291,
                293,290,330, 331, 334, 340,341,342, 343,344, 345,346,347,348,349, 
                351, 352,353, 354,361, 362,363, 372, 386, 703,984 ,975, 979]
imagenet = pd.read_csv("KD/Imagenet_SER_S9.csv", index_col="seq_id", names=c)
imagenet=imagenet[['c_' + str(i) for i in good_classes] + ['c2_' + str(i) for i in good_classes] + ['c3_' + str(i) for i in good_classes]]

In [4]:
everything = pd.read_csv("everything_merge.csv", index_col="seq_id")

In [5]:
# everything_s10 = pd.read_csv("KD/everything_SER_S10.csv", index_col="seq_id_buf")

In [6]:
# everything_s10.index.name='seq_id'

In [7]:
# len(everything), len(everything_s10), 

In [8]:
# everything['season']=9
# everything_s10['season']=10

In [9]:
# everything_2=pd.concat([everything, everything_s10])

In [10]:
# len(everything_2)

In [11]:
# join = everything_2.join(train_labels, how='left')
join = everything.join(train_labels, how='left')

In [12]:
join = imagenet.join(join, how='left')

In [13]:
# for model_id in ['pred_model_480_360_v2_as480_img1_', 'pred_model_480_360_v2_as480_img1_','pred_model_480_360_v2_as480_img1_',
#          'pred_model_480_360_v2_as512_img1_', 'pred_model_480_360_v2_as512_img2_','pred_model_480_360_v2_as512_img3_',
#          'pred_model_480_360_as480_img1_', 'pred_model_480_360_as480_img2_','pred_model_480_360_as480_img3_',
#          'pred_model_480_360_as512_img1_', 'pred_model_480_360_as512_img2_','pred_model_480_360_as512_img3_',
#           'pred_mean_back_2models_as512_','pred_back_as512_']:
#     join[model_id + 'nonempty'] = join[[model_id + c for c in list(train_labels.columns) if c != 'empty']].sum(axis=1)
#     join[model_id + 'maxnonempty'] = join[[model_id + c for c in list(train_labels.columns) if c != 'empty']].max(axis=1)

In [14]:
models = [
            'pred_model_480_360_as512_img',
            'pred_model_480_360_as480_img',
            'pred_model_480_360_v2_as512_img', 
            'pred_model_480_360_v2_as480_img', 
            'pred_model_512_384_v2_as512_img'
         ]
models2 = [
           'pred_mean_back_2models_as512_',
           'pred_mean_back_2models_as480_',
          ]
# 0.0031143950402571195
# 0.002732702686304169
for c in list(train_labels.columns):
    print(c)
    join['median_img1_' + c] = join[[m + '1_' + c for m in models]].median(axis=1)
    join['median_img2_' + c]=join['median_img1_' + c]
    join['median_img3_' + c]=join['median_img1_' + c]
    join.loc[join.count_imgs > 1, 'median_img2_' + c] = join.loc[join.count_imgs > 1, [m + '2_' + c for m in models]].median(axis=1)
    join.loc[join.count_imgs > 2, 'median_img3_' + c] = join.loc[join.count_imgs > 2, [m + '3_' + c for m in models]].median(axis=1)
    join['mean_imgs_' + c] = (join['median_img1_' + c] + join['median_img2_' + c]*0.8 + join['median_img3_' + c]*0.5) /2.3
    
    join['mean_combines_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs > 1, 'mean_combines_' + c] = join.loc[join.count_imgs > 1,[m  + c for m in models2]].median(axis=1)
    join['mean_with_combines_imgs_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs == 2, 'mean_with_combines_imgs_' + c]= join.loc[join.count_imgs == 2,'mean_imgs_' + c]*0.6 + join.loc[join.count_imgs == 2, 'mean_combines_' + c]*0.4
    join.loc[join.count_imgs > 2, 'mean_with_combines_imgs_' + c]= join.loc[join.count_imgs > 2,'mean_imgs_' + c]*0.2 + join.loc[join.count_imgs > 2, 'mean_combines_' + c]*0.8


aardvark
aardwolf
baboon
bat
batearedfox
buffalo
bushbuck
caracal
cattle
cheetah
civet
dikdik
duiker
eland
elephant
empty
gazellegrants
gazellethomsons
genet
giraffe
guineafowl
hare
hartebeest
hippopotamus
honeybadger
hyenaspotted
hyenastriped
impala
insectspider
jackal
koribustard
leopard
lionfemale
lionmale
mongoose
monkeyvervet
ostrich
otherbird
porcupine
reedbuck
reptiles
rhinoceros
rodents
secretarybird
serval
steenbok
topi
vulture
warthog
waterbuck
wildcat
wildebeest
zebra
zorilla


In [15]:
DATA_PATH = Path("../../datasets/wildlife") 
train_metadata = pd.read_csv(DATA_PATH / "train_metadata.csv", parse_dates= ['datetime'], index_col="seq_id")
train_metadata = train_metadata.groupby('seq_id').first()
train_metadata['month']=train_metadata.datetime.dt.month
train_metadata['hour']=train_metadata.datetime.dt.hour
join = join.join(train_metadata.hour, how='left')
join = join.join(train_metadata.month, how='left')

In [16]:

test_index = set(join.index[::10])
test_dataset     = join[ join.index.isin(test_index)]
trainval_dataset = join[~join.index.isin(test_index)]

val_index = set(trainval_dataset.index[::8])
val_dataset   = trainval_dataset[ trainval_dataset.index.isin(val_index)]
train_dataset = trainval_dataset[~trainval_dataset.index.isin(val_index)]
animal_columns =list(train_labels.columns)
animal_columns.remove('empty')



In [17]:
# test_dataset=join.copy()
# for c in list(train_labels.columns):
#     test_dataset['ens_img1_' + c] = (test_dataset['pred_model_480_360_v2_as480_img1_' + c] + test_dataset['pred_model_480_360_as480_img1_' + c])/2.0
#     test_dataset['ens_img2_' + c] = (test_dataset['pred_model_480_360_v2_as480_img2_' + c] + test_dataset['pred_model_480_360_as480_img2_' + c])/2.0
#     test_dataset['ens_img3_' + c] = (test_dataset['pred_model_480_360_v2_as480_img3_' + c] + test_dataset['pred_model_480_360_as480_img3_' + c])/2.0
    
#     test_dataset.loc[test_dataset.count_imgs < 3, 'ens_img3_' + c]=-999
#     test_dataset.loc[test_dataset.count_imgs < 2, 'ens_img2_' + c]=-999
#     test_dataset['ens_' + c] = test_dataset['ens_img1_' + c]
#     test_dataset.loc[test_dataset.count_imgs == 2, 'ens_' + c] = (test_dataset.loc[test_dataset.count_imgs == 2, 'ens_img1_' + c] +
#                                                   test_dataset.loc[test_dataset.count_imgs == 2, 'ens_img2_' + c]*0.9) /1.9
#     test_dataset.loc[test_dataset.count_imgs > 2, 'ens_' + c] = (test_dataset.loc[test_dataset.count_imgs > 2, 'ens_img1_' + c] +
#                                                  test_dataset.loc[test_dataset.count_imgs > 2, 'ens_img2_' +c]*0.8 +
#                                                  test_dataset.loc[test_dataset.count_imgs > 2, 'ens_img3_' + c]*0.5) / 2.3


# test_dataset.loc[test_dataset.count_imgs==2,'ens_empty'] = (test_dataset.loc[test_dataset.count_imgs==2,'ens_empty']*0.6 +test_dataset.loc[test_dataset.count_imgs==2,'pred_mean_back_2models_as480_empty']*0.4)
# test_dataset.loc[test_dataset.count_imgs>2,'ens_empty'] = (test_dataset.loc[test_dataset.count_imgs>2,'ens_empty']*0.1 +test_dataset.loc[test_dataset.count_imgs>2,'pred_mean_back_2models_as480_empty']*0.9)

# # for c in list(train_labels.columns):
#     test_dataset['ens_' + c]=test_dataset['pred_model_480_360_v2_as512_img1_' + c]
#     test_dataset.loc[test_dataset.count_imgs>1,'ens_' + c] = test_dataset.loc[test_dataset.count_imgs>1,'pred_mean_back_2models_as480_'+c]
#     test_dataset.loc[test_dataset.count_imgs>1,'ens_' + c] = test_dataset.loc[test_dataset.count_imgs>1,'pred_mean_back_2models_as512_'+c]
#     test_dataset.loc[test_dataset.count_imgs>1,'ens_' + c] = test_dataset.loc[test_dataset.count_imgs>1,'pred_mean_back_6channels_as512_'+c]
#     test_dataset.loc[test_dataset.count_imgs>1,'ens_' + c] = test_dataset.loc[test_dataset.count_imgs>1,'pred_mean_as480_'+c]





In [18]:
from sklearn.metrics import log_loss
loss = 0
for c in list(train_labels.columns):
    x = test_dataset[['mean_with_combines_imgs_' + c, c]].values
    if sum(x[:,1]) !=0:
#         print(c)
        l = log_loss(x[:,1], x[:,0])
        loss+=l
        print(c, l)
print(loss/54)
# 0.00659760884180274 img1
# 0.006059572723197153 img123
# 0.0045413404802776185 img123 with bg
# 0.0045249732546423236 img123 with bg + magic

# 0.003984543177212998


aardvark 0.00015580593593173252
aardwolf 0.00043686695366599345
baboon 0.0019937593201696997
batearedfox 0.0006326881809440662
buffalo 0.004529172958845754
bushbuck 0.0002558406002400329
caracal 0.00018053745229323456
cheetah 0.0010430752292739972
civet 3.429004737038826e-05
dikdik 0.0022225974171001498
duiker 0.0008234743219526635
eland 0.0008848665785864957
elephant 0.004065639460409693
empty 0.03849730233659921
gazellegrants 0.006319377561592043
gazellethomsons 0.015496168045458825
genet 0.00042773152465165495
giraffe 0.0020826122270030225
guineafowl 0.001212648155267016
hare 0.0005475074010940107
hartebeest 0.002524939809517865
hippopotamus 0.0008965207038867278
honeybadger 0.00012688721491628402
hyenaspotted 0.002986075647900426
hyenastriped 0.00019352960643269238
impala 0.005292393517757406
insectspider 0.003251063312309871
jackal 0.0012640419092682933
koribustard 0.0009710234502126199
leopard 0.0004357800095262185
lionfemale 0.0031450014792266234
lionmale 0.0012976665124839722
m

In [ ]:
aardvark 0.00012400494331310212
aardwolf 0.00041863614940684037
baboon 0.0015140601225253952
batearedfox 0.0005680056603519342
buffalo 0.0038392584692827085
bushbuck 0.00019572546624137754
caracal 0.0001380608397927396
cheetah 0.0013401865619093502
civet 2.990007267168342e-05
dikdik 0.0016899371254598748
duiker 0.0009156356669827548
eland 0.001496580786512857
elephant 0.0035461047087422836
empty 0.036948739703100746
gazellegrants 0.0065368292214597395
gazellethomsons 0.015628731321237696
genet 0.00032101311571715324
giraffe 0.0018571718499794357
guineafowl 0.0010298060151850323
hare 0.0004212891333290041
hartebeest 0.0028648895539727377
hippopotamus 0.0006920607192706828
honeybadger 9.990881401212624e-05
hyenaspotted 0.0028974383025049876
hyenastriped 0.0002566949206167334
impala 0.004393419582901715
insectspider 0.0028657092706877046
jackal 0.0009919161656297038
koribustard 0.0010793613811273277
leopard 0.0003318864758598873
lionfemale 0.002785041606506931
lionmale 0.0015079438245437524
mongoose 0.0003256953918198407
monkeyvervet 0.00045379925536576256
ostrich 0.0016053229450361305
otherbird 0.010406196166245983
porcupine 0.00015614320082107776
reedbuck 0.001765725874393598
reptiles 0.0005382602768566637
secretarybird 0.0009860670624208204
serval 0.00028550213482042455
steenbok 0.00045281040134754905
topi 0.0020726321662736446
vulture 0.00020933188758334487
warthog 0.0041312621764125574
waterbuck 0.00012147940954261093
wildebeest 0.016439716260575107
zebra 0.015226105813035731
0.0028611481106182745

In [19]:
feature_classes=list(train_labels.columns)

# feature_classes.remove('reptiles')
# feature_classes.remove('caracal')
# feature_classes.remove('wildcat')
# feature_classes.remove('honeybadger')
# feature_classes.remove('vulture')
# feature_classes.remove('duiker')
# feature_classes.remove('civet')
# feature_classes.remove('genet')
# feature_classes.remove('rhinoceros')
# feature_classes.remove('zorilla')
# feature_classes.remove('cattle')
# feature_classes.remove('steenbok')
# feature_classes.remove('bat')

# feature_classes=['zebra', 'empty']
# feature_classes.append('nonempty')
# feature_classes.append('maxnonempty')


    
num_leaves={
    'empty': 20,

    
}
max_depth={
    'gazellethomsons': 3,


}
learning_rate={
#     'gazellethomsons': 0.001,
    'empty':0.008,
}



# ['pred_mean_back_2models_as512_' + c for c in feature_classes] +\
# ['pred_mean_back_6channels_as512_' + c for c in feature_classes] +\
# ['pred_model_512_384_v2_as480_img1_' + c for c in feature_classes] +\
# ['pred_model_512_384_v2_as480_img2_' + c for c in feature_classes] +\
# # ['pred_model_512_384_v2_as480_img3_' + c for c in feature_classes] +\

# ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
# ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
# ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\

# ['pred_back_as512_' + c for c in feature_classes] +\ 109->108
# ['pred_mean_as512_' + c for c in feature_classes] +\

# pred_back_as480_ +\ 108-> 110
# pred_mean_as480_ +\ 110 -> 113



# ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
# ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
# ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\

# ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\






columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
['pred_mean_back_2models_as480_' + c for c in feature_classes] +\
['pred_mean_back_2models_as512_' + c for c in feature_classes] +\
['pred_back_as480_' + c for c in feature_classes] +\
['pred_back_as512_' + c for c in feature_classes] +\
['median_img1_' + c for c in feature_classes]+\
['median_img2_' + c for c in feature_classes]+\
['median_img3_' + c for c in feature_classes]+\
['mean_imgs_' + c for c in feature_classes]+\
['mean_combines_' + c for c in feature_classes]+\
['mean_with_combines_imgs_' + c for c in feature_classes]+\
['count_imgs', 'hour', 'month']





# ['pred_back_as512_' + c for c in feature_classes] 
# ['count_imgs']

# ['pred_mean_as512_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
# ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
# ['pred_model_512_384_v2_as480_img1_' + c for c in feature_classes] +\
# ['pred_model_512_384_v2_as480_img2_' + c for c in feature_classes] +\
# ['pred_model_512_384_v2_as480_img3_' + c for c in feature_classes] +\










In [27]:
clazz='wildebeest'

X_train = train_dataset[columns]
y_train = train_dataset[clazz]
X_val = val_dataset[columns]
y_val = val_dataset[clazz]

In [28]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# clf = GaussianNB()
# clf.fit(X_train, y_train)
# X_test = test_dataset[columns]
# y_test = test_dataset[clazz]

# print(clazz, log_loss(y_test, clf.predict_proba(X_test.values)[:,1]))

In [29]:
import xgboost as xgb

In [30]:
# clf = xgb.XGBClassifier(max_depth=4, 
#                         n_estimators=5000,
#                         learning_rate=0.005,
#                         objective='binary:logistic',
                        
#                        eval_metric='logloss')

# gbm_xgb=clf.fit(X_train, y_train, eval_set = [(X_val, y_val)],early_stopping_rounds=20,verbose=True)


In [31]:
# X_test = test_dataset[columns]
# y_test = test_dataset[clazz]

# print(clazz, log_loss(y_test, gbm.predict(X_test.values)))

In [32]:
# X_test = test_dataset[columns]
# y_test = test_dataset[clazz]

# print(clazz, log_loss(y_test.values, (gbm_xgb.predict_proba(X_test)[:,1])))


In [33]:


lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

scale = y_train.mean()
print(math.log(1/scale))
params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'binary_logloss'},
            'num_leaves': 20,
            'max_depth': max_depth.get(clazz, -1),
            'learning_rate': learning_rate.get(clazz, 0.01/math.log(1/scale)),
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0,
    #         'scale_pos_weight':scale_pos_weight.get(clazz, scale)
            'scale_pos_weight':1

}


gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=10000,
                    valid_sets=lgb_val,
                    early_stopping_rounds=50)
gbm.save_model("lgbs_v6/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

X_test = test_dataset[columns]
y_test = test_dataset[clazz]

print(clazz, log_loss(y_test, gbm.predict(X_test.values)))
#9s
# [2300]	valid_0's binary_logloss: 0.0291195
# empty 0.02906960726224346  vs 0.029772033801938245   

# 9 + 10 se
# [2384]	valid_0's binary_logloss: 0.02822
# empty 0.029531312794916133
# empty 0.02883884781385263 precompute

# [2533]	valid_0's binary_logloss: 0.0121197
# zebra 0.01133470250264239 vs 0.0119620473182895

# fold2 mean topi 0.0024336523264832923 model topi 0.0018745894933088797

# jackal 0.0008312343284820714

# [2449]	valid_0's binary_logloss: 0.0145814
# wildebeest 0.014376435255919027

2.663394994717997
[1]	valid_0's binary_logloss: 0.251796
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.248445
[3]	valid_0's binary_logloss: 0.245245
[4]	valid_0's binary_logloss: 0.242186
[5]	valid_0's binary_logloss: 0.239256
[6]	valid_0's binary_logloss: 0.236439
[7]	valid_0's binary_logloss: 0.23373
[8]	valid_0's binary_logloss: 0.231118
[9]	valid_0's binary_logloss: 0.228597
[10]	valid_0's binary_logloss: 0.226166
[11]	valid_0's binary_logloss: 0.223813
[12]	valid_0's binary_logloss: 0.221532
[13]	valid_0's binary_logloss: 0.219323
[14]	valid_0's binary_logloss: 0.217178
[15]	valid_0's binary_logloss: 0.215097
[16]	valid_0's binary_logloss: 0.213068
[17]	valid_0's binary_logloss: 0.211097
[18]	valid_0's binary_logloss: 0.209175
[19]	valid_0's binary_logloss: 0.207298
[20]	valid_0's binary_logloss: 0.205474
[21]	valid_0's binary_logloss: 0.203686
[22]	valid_0's binary_logloss: 0.20194
[23]	valid_0's binary_logloss: 0.200235
[24]	valid_0

[202]	valid_0's binary_logloss: 0.0813419
[203]	valid_0's binary_logloss: 0.0810381
[204]	valid_0's binary_logloss: 0.0807366
[205]	valid_0's binary_logloss: 0.0804356
[206]	valid_0's binary_logloss: 0.080138
[207]	valid_0's binary_logloss: 0.0798422
[208]	valid_0's binary_logloss: 0.0795482
[209]	valid_0's binary_logloss: 0.0792555
[210]	valid_0's binary_logloss: 0.0789636
[211]	valid_0's binary_logloss: 0.0786746
[212]	valid_0's binary_logloss: 0.0783866
[213]	valid_0's binary_logloss: 0.0781
[214]	valid_0's binary_logloss: 0.0778164
[215]	valid_0's binary_logloss: 0.0775327
[216]	valid_0's binary_logloss: 0.0772516
[217]	valid_0's binary_logloss: 0.0769708
[218]	valid_0's binary_logloss: 0.0766929
[219]	valid_0's binary_logloss: 0.0764147
[220]	valid_0's binary_logloss: 0.0761394
[221]	valid_0's binary_logloss: 0.0758653
[222]	valid_0's binary_logloss: 0.075592
[223]	valid_0's binary_logloss: 0.0753208
[224]	valid_0's binary_logloss: 0.0750503
[225]	valid_0's binary_logloss: 0.07478

[398]	valid_0's binary_logloss: 0.0437046
[399]	valid_0's binary_logloss: 0.0435885
[400]	valid_0's binary_logloss: 0.0434725
[401]	valid_0's binary_logloss: 0.0433561
[402]	valid_0's binary_logloss: 0.0432407
[403]	valid_0's binary_logloss: 0.0431244
[404]	valid_0's binary_logloss: 0.0430095
[405]	valid_0's binary_logloss: 0.0428959
[406]	valid_0's binary_logloss: 0.0427816
[407]	valid_0's binary_logloss: 0.0426682
[408]	valid_0's binary_logloss: 0.0425554
[409]	valid_0's binary_logloss: 0.0424426
[410]	valid_0's binary_logloss: 0.0423304
[411]	valid_0's binary_logloss: 0.0422194
[412]	valid_0's binary_logloss: 0.042109
[413]	valid_0's binary_logloss: 0.0419974
[414]	valid_0's binary_logloss: 0.0418872
[415]	valid_0's binary_logloss: 0.0417778
[416]	valid_0's binary_logloss: 0.041669
[417]	valid_0's binary_logloss: 0.0415608
[418]	valid_0's binary_logloss: 0.0414536
[419]	valid_0's binary_logloss: 0.0413465
[420]	valid_0's binary_logloss: 0.0412407
[421]	valid_0's binary_logloss: 0.04

[594]	valid_0's binary_logloss: 0.0280916
[595]	valid_0's binary_logloss: 0.0280409
[596]	valid_0's binary_logloss: 0.0279907
[597]	valid_0's binary_logloss: 0.0279395
[598]	valid_0's binary_logloss: 0.0278888
[599]	valid_0's binary_logloss: 0.0278386
[600]	valid_0's binary_logloss: 0.0277887
[601]	valid_0's binary_logloss: 0.0277384
[602]	valid_0's binary_logloss: 0.0276882
[603]	valid_0's binary_logloss: 0.0276381
[604]	valid_0's binary_logloss: 0.0275883
[605]	valid_0's binary_logloss: 0.0275385
[606]	valid_0's binary_logloss: 0.0274889
[607]	valid_0's binary_logloss: 0.0274407
[608]	valid_0's binary_logloss: 0.027392
[609]	valid_0's binary_logloss: 0.0273431
[610]	valid_0's binary_logloss: 0.0272958
[611]	valid_0's binary_logloss: 0.0272486
[612]	valid_0's binary_logloss: 0.0271998
[613]	valid_0's binary_logloss: 0.0271517
[614]	valid_0's binary_logloss: 0.0271042
[615]	valid_0's binary_logloss: 0.0270573
[616]	valid_0's binary_logloss: 0.0270093
[617]	valid_0's binary_logloss: 0.0

[790]	valid_0's binary_logloss: 0.0210498
[791]	valid_0's binary_logloss: 0.0210262
[792]	valid_0's binary_logloss: 0.0210027
[793]	valid_0's binary_logloss: 0.020979
[794]	valid_0's binary_logloss: 0.0209558
[795]	valid_0's binary_logloss: 0.0209324
[796]	valid_0's binary_logloss: 0.0209098
[797]	valid_0's binary_logloss: 0.020887
[798]	valid_0's binary_logloss: 0.0208645
[799]	valid_0's binary_logloss: 0.0208429
[800]	valid_0's binary_logloss: 0.0208209
[801]	valid_0's binary_logloss: 0.0207983
[802]	valid_0's binary_logloss: 0.0207758
[803]	valid_0's binary_logloss: 0.0207519
[804]	valid_0's binary_logloss: 0.0207296
[805]	valid_0's binary_logloss: 0.0207072
[806]	valid_0's binary_logloss: 0.0206834
[807]	valid_0's binary_logloss: 0.0206621
[808]	valid_0's binary_logloss: 0.0206391
[809]	valid_0's binary_logloss: 0.0206182
[810]	valid_0's binary_logloss: 0.0205969
[811]	valid_0's binary_logloss: 0.020575
[812]	valid_0's binary_logloss: 0.0205534
[813]	valid_0's binary_logloss: 0.020

...

[1560]	valid_0's binary_logloss: 0.0150944
[1561]	valid_0's binary_logloss: 0.0150927
[1562]	valid_0's binary_logloss: 0.0150913
[1563]	valid_0's binary_logloss: 0.0150896
[1564]	valid_0's binary_logloss: 0.0150881
[1565]	valid_0's binary_logloss: 0.0150866
[1566]	valid_0's binary_logloss: 0.0150852
[1567]	valid_0's binary_logloss: 0.0150836
[1568]	valid_0's binary_logloss: 0.0150825
[1569]	valid_0's binary_logloss: 0.0150818
[1570]	valid_0's binary_logloss: 0.0150815
[1571]	valid_0's binary_logloss: 0.0150805
[1572]	valid_0's binary_logloss: 0.0150788
[1573]	valid_0's binary_logloss: 0.0150773
[1574]	valid_0's binary_logloss: 0.0150761
[1575]	valid_0's binary_logloss: 0.0150744
[1576]	valid_0's binary_logloss: 0.0150725
[1577]	valid_0's binary_logloss: 0.0150698
[1578]	valid_0's binary_logloss: 0.0150683
[1579]	valid_0's binary_logloss: 0.0150661
[1580]	valid_0's binary_logloss: 0.0150655
[1581]	valid_0's binary_logloss: 0.0150634
[1582]	valid_0's binary_logloss: 0.0150621
[1583]	vali

[1752]	valid_0's binary_logloss: 0.014878
[1753]	valid_0's binary_logloss: 0.0148774
[1754]	valid_0's binary_logloss: 0.0148772
[1755]	valid_0's binary_logloss: 0.0148759
[1756]	valid_0's binary_logloss: 0.0148751
[1757]	valid_0's binary_logloss: 0.0148743
[1758]	valid_0's binary_logloss: 0.0148741
[1759]	valid_0's binary_logloss: 0.0148727
[1760]	valid_0's binary_logloss: 0.0148718
[1761]	valid_0's binary_logloss: 0.0148708
[1762]	valid_0's binary_logloss: 0.01487
[1763]	valid_0's binary_logloss: 0.0148689
[1764]	valid_0's binary_logloss: 0.0148683
[1765]	valid_0's binary_logloss: 0.0148676
[1766]	valid_0's binary_logloss: 0.0148657
[1767]	valid_0's binary_logloss: 0.0148636
[1768]	valid_0's binary_logloss: 0.0148621
[1769]	valid_0's binary_logloss: 0.0148602
[1770]	valid_0's binary_logloss: 0.0148583
[1771]	valid_0's binary_logloss: 0.0148582
[1772]	valid_0's binary_logloss: 0.0148572
[1773]	valid_0's binary_logloss: 0.0148561
[1774]	valid_0's binary_logloss: 0.0148549
[1775]	valid_0

[1944]	valid_0's binary_logloss: 0.0147586
[1945]	valid_0's binary_logloss: 0.0147565
[1946]	valid_0's binary_logloss: 0.0147565
[1947]	valid_0's binary_logloss: 0.0147557
[1948]	valid_0's binary_logloss: 0.0147543
[1949]	valid_0's binary_logloss: 0.0147529
[1950]	valid_0's binary_logloss: 0.0147526
[1951]	valid_0's binary_logloss: 0.0147523
[1952]	valid_0's binary_logloss: 0.0147518
[1953]	valid_0's binary_logloss: 0.0147508
[1954]	valid_0's binary_logloss: 0.0147497
[1955]	valid_0's binary_logloss: 0.0147489
[1956]	valid_0's binary_logloss: 0.0147488
[1957]	valid_0's binary_logloss: 0.0147484
[1958]	valid_0's binary_logloss: 0.0147475
[1959]	valid_0's binary_logloss: 0.0147475
[1960]	valid_0's binary_logloss: 0.0147469
[1961]	valid_0's binary_logloss: 0.0147468
[1962]	valid_0's binary_logloss: 0.0147451
[1963]	valid_0's binary_logloss: 0.0147434
[1964]	valid_0's binary_logloss: 0.0147434
[1965]	valid_0's binary_logloss: 0.0147416
[1966]	valid_0's binary_logloss: 0.0147415
[1967]	vali

[2135]	valid_0's binary_logloss: 0.0146718
[2136]	valid_0's binary_logloss: 0.0146711
[2137]	valid_0's binary_logloss: 0.014671
[2138]	valid_0's binary_logloss: 0.0146702
[2139]	valid_0's binary_logloss: 0.01467
[2140]	valid_0's binary_logloss: 0.0146702
[2141]	valid_0's binary_logloss: 0.01467
[2142]	valid_0's binary_logloss: 0.0146697
[2143]	valid_0's binary_logloss: 0.0146689
[2144]	valid_0's binary_logloss: 0.0146685
[2145]	valid_0's binary_logloss: 0.0146682
[2146]	valid_0's binary_logloss: 0.0146677
[2147]	valid_0's binary_logloss: 0.0146667
[2148]	valid_0's binary_logloss: 0.0146656
[2149]	valid_0's binary_logloss: 0.0146645
[2150]	valid_0's binary_logloss: 0.0146639
[2151]	valid_0's binary_logloss: 0.0146635
[2152]	valid_0's binary_logloss: 0.014663
[2153]	valid_0's binary_logloss: 0.0146627
[2154]	valid_0's binary_logloss: 0.0146631
[2155]	valid_0's binary_logloss: 0.014663
[2156]	valid_0's binary_logloss: 0.0146638
[2157]	valid_0's binary_logloss: 0.0146645
[2158]	valid_0's b

[2327]	valid_0's binary_logloss: 0.0146277
[2328]	valid_0's binary_logloss: 0.0146288
[2329]	valid_0's binary_logloss: 0.0146286
[2330]	valid_0's binary_logloss: 0.0146296
[2331]	valid_0's binary_logloss: 0.0146286
[2332]	valid_0's binary_logloss: 0.0146276
[2333]	valid_0's binary_logloss: 0.0146269
[2334]	valid_0's binary_logloss: 0.0146259
[2335]	valid_0's binary_logloss: 0.0146253
[2336]	valid_0's binary_logloss: 0.0146253
[2337]	valid_0's binary_logloss: 0.0146255
[2338]	valid_0's binary_logloss: 0.0146252
[2339]	valid_0's binary_logloss: 0.0146253
[2340]	valid_0's binary_logloss: 0.0146255
[2341]	valid_0's binary_logloss: 0.0146249
[2342]	valid_0's binary_logloss: 0.0146239
[2343]	valid_0's binary_logloss: 0.0146232
[2344]	valid_0's binary_logloss: 0.0146227
[2345]	valid_0's binary_logloss: 0.0146219
[2346]	valid_0's binary_logloss: 0.0146215
[2347]	valid_0's binary_logloss: 0.0146207
[2348]	valid_0's binary_logloss: 0.0146208
[2349]	valid_0's binary_logloss: 0.0146207
[2350]	vali

In [27]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importance(),X_test.columns)), columns=['Value','Feature'])
feature_imp

,Value,Feature
0,0,count_imgs
1,0,mean_combines_aardvark
2,0,mean_combines_aardwolf
3,0,mean_combines_baboon
4,0,mean_combines_batearedfox
...,...,...
1186,45,month
1187,45,pred_model_480_360_as512_img1_gazellethomsons
1188,47,median_img3_gazellethomsons
1189,51,mean_with_combines_imgs_gazellethomsons


In [17]:
clazzs = ['baboon', 'buffalo',  'elephant',  'empty', 'gazellegrants', 'gazellethomsons', 'giraffe', 'guineafowl',
                    'hare', 'hartebeest', 'hyenaspotted', 'honeybadger',  'impala', 'insectspider', 'mongoose', 'otherbird',
                    'warthog', 'zebra', 'wildebeest']
for clazz in clazzs:
    print(clazz)
    X_train = train_dataset[columns]
    y_train = train_dataset[clazz]
    if sum(y_train) < 5:
        continue
    X_val = val_dataset[columns]
    y_val = val_dataset[clazz]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

    scale = y_train.mean()
    if scale < 0.5:
        scale=math.log(1/scale)
    else:
        scale=math.sqrt(scale)
    print(scale)
    params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'binary_logloss'},
            'num_leaves': num_leaves.get(clazz, 20),
            'max_depth': max_depth.get(clazz, -1),
            'learning_rate': learning_rate.get(clazz, 0.01/scale),
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0,
    #         'scale_pos_weight':scale_pos_weight.get(clazz, scale)
            'scale_pos_weight':1

    }


    gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=10000,
                        valid_sets=lgb_val,
                        early_stopping_rounds=50)
    gbm.save_model("lgbs_v6/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

    X_test = test_dataset[columns]
    y_test = test_dataset[clazz]
    if sum(y_test) ==0:
        print(clazz)
    else:
        print(clazz, log_loss(y_test, gbm.predict(X_test.values)))

baboon
6.734840502166809
[1]	valid_0's binary_logloss: 0.00759944
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.00726848
[3]	valid_0's binary_logloss: 0.00703148
[4]	valid_0's binary_logloss: 0.00683573
[5]	valid_0's binary_logloss: 0.00669602
[6]	valid_0's binary_logloss: 0.00655415
[7]	valid_0's binary_logloss: 0.00643993
[8]	valid_0's binary_logloss: 0.00633942
[9]	valid_0's binary_logloss: 0.00624734
[10]	valid_0's binary_logloss: 0.00616657
[11]	valid_0's binary_logloss: 0.00608405
[12]	valid_0's binary_logloss: 0.00601169
[13]	valid_0's binary_logloss: 0.00594256
[14]	valid_0's binary_logloss: 0.00588165
[15]	valid_0's binary_logloss: 0.00582109
[16]	valid_0's binary_logloss: 0.00576563
[17]	valid_0's binary_logloss: 0.00571005
[18]	valid_0's binary_logloss: 0.00565935
[19]	valid_0's binary_logloss: 0.00561224
[20]	valid_0's binary_logloss: 0.00556905
[21]	valid_0's binary_logloss: 0.00552565
[22]	valid_0's binary_logloss: 0.00548305

[193]	valid_0's binary_logloss: 0.00334538
[194]	valid_0's binary_logloss: 0.00334064
[195]	valid_0's binary_logloss: 0.0033358
[196]	valid_0's binary_logloss: 0.00333
[197]	valid_0's binary_logloss: 0.00332442
[198]	valid_0's binary_logloss: 0.00331929
[199]	valid_0's binary_logloss: 0.00331349
[200]	valid_0's binary_logloss: 0.00330771
[201]	valid_0's binary_logloss: 0.00330231
[202]	valid_0's binary_logloss: 0.00329682
[203]	valid_0's binary_logloss: 0.00329156
[204]	valid_0's binary_logloss: 0.00328632
[205]	valid_0's binary_logloss: 0.00328111
[206]	valid_0's binary_logloss: 0.003276
[207]	valid_0's binary_logloss: 0.00327056
[208]	valid_0's binary_logloss: 0.00326523
[209]	valid_0's binary_logloss: 0.00325985
[210]	valid_0's binary_logloss: 0.00325421
[211]	valid_0's binary_logloss: 0.00324905
[212]	valid_0's binary_logloss: 0.00324379
[213]	valid_0's binary_logloss: 0.00323885
[214]	valid_0's binary_logloss: 0.00323396
[215]	valid_0's binary_logloss: 0.00322816
[216]	valid_0's b

[386]	valid_0's binary_logloss: 0.00258017
[387]	valid_0's binary_logloss: 0.00257743
[388]	valid_0's binary_logloss: 0.00257486
[389]	valid_0's binary_logloss: 0.0025723
[390]	valid_0's binary_logloss: 0.00256997
[391]	valid_0's binary_logloss: 0.00256694
[392]	valid_0's binary_logloss: 0.00256412
[393]	valid_0's binary_logloss: 0.00256142
[394]	valid_0's binary_logloss: 0.00255856
[395]	valid_0's binary_logloss: 0.00255567
[396]	valid_0's binary_logloss: 0.00255302
[397]	valid_0's binary_logloss: 0.00255057
[398]	valid_0's binary_logloss: 0.0025481
[399]	valid_0's binary_logloss: 0.00254568
[400]	valid_0's binary_logloss: 0.00254318
[401]	valid_0's binary_logloss: 0.00254045
[402]	valid_0's binary_logloss: 0.00253748
[403]	valid_0's binary_logloss: 0.00253485
[404]	valid_0's binary_logloss: 0.00253197
[405]	valid_0's binary_logloss: 0.00252905
[406]	valid_0's binary_logloss: 0.002526
[407]	valid_0's binary_logloss: 0.00252308
[408]	valid_0's binary_logloss: 0.00252005
[409]	valid_0's

[578]	valid_0's binary_logloss: 0.002155
[579]	valid_0's binary_logloss: 0.00215309
[580]	valid_0's binary_logloss: 0.00215129
[581]	valid_0's binary_logloss: 0.00214914
[582]	valid_0's binary_logloss: 0.00214724
[583]	valid_0's binary_logloss: 0.00214521
[584]	valid_0's binary_logloss: 0.00214329
[585]	valid_0's binary_logloss: 0.00214128
[586]	valid_0's binary_logloss: 0.0021397
[587]	valid_0's binary_logloss: 0.00213814
[588]	valid_0's binary_logloss: 0.00213648
[589]	valid_0's binary_logloss: 0.0021349
[590]	valid_0's binary_logloss: 0.00213335
[591]	valid_0's binary_logloss: 0.00213166
[592]	valid_0's binary_logloss: 0.00212996
[593]	valid_0's binary_logloss: 0.00212826
[594]	valid_0's binary_logloss: 0.00212657
[595]	valid_0's binary_logloss: 0.00212485
[596]	valid_0's binary_logloss: 0.00212327
[597]	valid_0's binary_logloss: 0.00212183
[598]	valid_0's binary_logloss: 0.00212029
[599]	valid_0's binary_logloss: 0.00211874
[600]	valid_0's binary_logloss: 0.00211721
[601]	valid_0's

[772]	valid_0's binary_logloss: 0.00186817
[773]	valid_0's binary_logloss: 0.00186707
[774]	valid_0's binary_logloss: 0.00186615
[775]	valid_0's binary_logloss: 0.00186511
[776]	valid_0's binary_logloss: 0.00186414
[777]	valid_0's binary_logloss: 0.00186306
[778]	valid_0's binary_logloss: 0.00186209
[779]	valid_0's binary_logloss: 0.00186113
[780]	valid_0's binary_logloss: 0.00186019
[781]	valid_0's binary_logloss: 0.00185897
[782]	valid_0's binary_logloss: 0.00185784
[783]	valid_0's binary_logloss: 0.00185681
[784]	valid_0's binary_logloss: 0.00185578
[785]	valid_0's binary_logloss: 0.00185475
[786]	valid_0's binary_logloss: 0.00185351
[787]	valid_0's binary_logloss: 0.00185218
[788]	valid_0's binary_logloss: 0.00185106
[789]	valid_0's binary_logloss: 0.00184968
[790]	valid_0's binary_logloss: 0.00184834
[791]	valid_0's binary_logloss: 0.001847
[792]	valid_0's binary_logloss: 0.00184572
[793]	valid_0's binary_logloss: 0.00184438
[794]	valid_0's binary_logloss: 0.00184302
[795]	valid_0

...

[2670]	valid_0's binary_logloss: 0.0140166
[2671]	valid_0's binary_logloss: 0.0140164
[2672]	valid_0's binary_logloss: 0.0140161
[2673]	valid_0's binary_logloss: 0.0140157
[2674]	valid_0's binary_logloss: 0.0140153
[2675]	valid_0's binary_logloss: 0.0140146
[2676]	valid_0's binary_logloss: 0.0140144
[2677]	valid_0's binary_logloss: 0.0140146
[2678]	valid_0's binary_logloss: 0.0140146
[2679]	valid_0's binary_logloss: 0.0140144
[2680]	valid_0's binary_logloss: 0.0140135
[2681]	valid_0's binary_logloss: 0.0140132
[2682]	valid_0's binary_logloss: 0.0140131
[2683]	valid_0's binary_logloss: 0.0140129
[2684]	valid_0's binary_logloss: 0.0140126
[2685]	valid_0's binary_logloss: 0.0140125
[2686]	valid_0's binary_logloss: 0.0140126
[2687]	valid_0's binary_logloss: 0.0140123
[2688]	valid_0's binary_logloss: 0.0140123
[2689]	valid_0's binary_logloss: 0.0140126
[2690]	valid_0's binary_logloss: 0.0140129
[2691]	valid_0's binary_logloss: 0.0140128
[2692]	valid_0's binary_logloss: 0.014013
[2693]	valid

[2861]	valid_0's binary_logloss: 0.0139821
[2862]	valid_0's binary_logloss: 0.0139822
[2863]	valid_0's binary_logloss: 0.0139828
[2864]	valid_0's binary_logloss: 0.0139825
[2865]	valid_0's binary_logloss: 0.0139827
[2866]	valid_0's binary_logloss: 0.013982
[2867]	valid_0's binary_logloss: 0.0139813
[2868]	valid_0's binary_logloss: 0.0139808
[2869]	valid_0's binary_logloss: 0.0139804
[2870]	valid_0's binary_logloss: 0.0139796
[2871]	valid_0's binary_logloss: 0.0139796
[2872]	valid_0's binary_logloss: 0.0139793
[2873]	valid_0's binary_logloss: 0.0139791
[2874]	valid_0's binary_logloss: 0.0139793
[2875]	valid_0's binary_logloss: 0.0139794
[2876]	valid_0's binary_logloss: 0.013979
[2877]	valid_0's binary_logloss: 0.0139787
[2878]	valid_0's binary_logloss: 0.0139787
[2879]	valid_0's binary_logloss: 0.0139785
[2880]	valid_0's binary_logloss: 0.013978
[2881]	valid_0's binary_logloss: 0.0139784
[2882]	valid_0's binary_logloss: 0.0139786
[2883]	valid_0's binary_logloss: 0.0139787
[2884]	valid_0

[3052]	valid_0's binary_logloss: 0.0139627
[3053]	valid_0's binary_logloss: 0.0139621
[3054]	valid_0's binary_logloss: 0.0139616
[3055]	valid_0's binary_logloss: 0.0139612
[3056]	valid_0's binary_logloss: 0.0139607
[3057]	valid_0's binary_logloss: 0.0139606
[3058]	valid_0's binary_logloss: 0.0139602
[3059]	valid_0's binary_logloss: 0.0139602
[3060]	valid_0's binary_logloss: 0.0139596
[3061]	valid_0's binary_logloss: 0.0139594
[3062]	valid_0's binary_logloss: 0.0139592
[3063]	valid_0's binary_logloss: 0.0139591
[3064]	valid_0's binary_logloss: 0.0139593
[3065]	valid_0's binary_logloss: 0.0139594
[3066]	valid_0's binary_logloss: 0.0139595
[3067]	valid_0's binary_logloss: 0.0139593
[3068]	valid_0's binary_logloss: 0.013959
[3069]	valid_0's binary_logloss: 0.0139587
[3070]	valid_0's binary_logloss: 0.0139578
[3071]	valid_0's binary_logloss: 0.0139575
[3072]	valid_0's binary_logloss: 0.0139575
[3073]	valid_0's binary_logloss: 0.0139572
[3074]	valid_0's binary_logloss: 0.0139568
[3075]	valid

[3244]	valid_0's binary_logloss: 0.0139377
[3245]	valid_0's binary_logloss: 0.013938
[3246]	valid_0's binary_logloss: 0.0139382
[3247]	valid_0's binary_logloss: 0.0139375
[3248]	valid_0's binary_logloss: 0.0139365
[3249]	valid_0's binary_logloss: 0.0139356
[3250]	valid_0's binary_logloss: 0.0139347
[3251]	valid_0's binary_logloss: 0.0139343
[3252]	valid_0's binary_logloss: 0.0139338
[3253]	valid_0's binary_logloss: 0.0139331
[3254]	valid_0's binary_logloss: 0.0139326
[3255]	valid_0's binary_logloss: 0.0139323
[3256]	valid_0's binary_logloss: 0.0139318
[3257]	valid_0's binary_logloss: 0.0139317
[3258]	valid_0's binary_logloss: 0.0139313
[3259]	valid_0's binary_logloss: 0.0139308
[3260]	valid_0's binary_logloss: 0.0139309
[3261]	valid_0's binary_logloss: 0.0139312
[3262]	valid_0's binary_logloss: 0.0139305
[3263]	valid_0's binary_logloss: 0.0139311
[3264]	valid_0's binary_logloss: 0.0139319
[3265]	valid_0's binary_logloss: 0.0139324
[3266]	valid_0's binary_logloss: 0.0139319
[3267]	valid

[3437]	valid_0's binary_logloss: 0.0139129
[3438]	valid_0's binary_logloss: 0.013913
[3439]	valid_0's binary_logloss: 0.0139129
[3440]	valid_0's binary_logloss: 0.013913
[3441]	valid_0's binary_logloss: 0.0139133
[3442]	valid_0's binary_logloss: 0.0139136
[3443]	valid_0's binary_logloss: 0.013913
[3444]	valid_0's binary_logloss: 0.0139124
[3445]	valid_0's binary_logloss: 0.0139127
[3446]	valid_0's binary_logloss: 0.0139127
[3447]	valid_0's binary_logloss: 0.0139125
[3448]	valid_0's binary_logloss: 0.0139124
[3449]	valid_0's binary_logloss: 0.0139114
[3450]	valid_0's binary_logloss: 0.0139112
[3451]	valid_0's binary_logloss: 0.0139112
[3452]	valid_0's binary_logloss: 0.0139107
[3453]	valid_0's binary_logloss: 0.0139103
[3454]	valid_0's binary_logloss: 0.0139099
[3455]	valid_0's binary_logloss: 0.0139094
[3456]	valid_0's binary_logloss: 0.0139099
[3457]	valid_0's binary_logloss: 0.0139098
[3458]	valid_0's binary_logloss: 0.0139094
[3459]	valid_0's binary_logloss: 0.0139087
[3460]	valid_0

In [61]:
X_test = test_dataset[columns]
ls = 0
c = list(train_labels.columns)
c.remove('reptiles')
import os
for clazz in c:
    is_model = False

    y_test = test_dataset[clazz]
    if sum(y_test) ==0 :
        print(clazz, 'bad')
        continue
    if not os.path.isfile("lgbs_v5/" + clazz + '.txt'):
        x = test_dataset['mean_with_combines_imgs_' + clazz].values
    else:
        try:
            model = lgb.Booster(model_file="lgbs_v5/" + clazz + '.txt')
            x=model.predict(X_test)
            is_model=True
        except:
            x = test_dataset['mean_with_combines_imgs_' + clazz].values

    
    l =log_loss(y_test,x )
    ls+=l
    print(clazz, l, is_model)
print(ls/54)

aardvark 0.00012400494331310212 False
aardwolf 0.00041863614940684037 False
baboon 0.0015140601225253952 False
bat bad
batearedfox 0.0005680056603519342 False
buffalo 0.0038392584692827085 False
bushbuck 0.00019572546624137754 False
caracal 0.0001380608397927396 False
cattle bad
cheetah 0.0013401865619093502 False
civet 2.990007267168342e-05 False
dikdik 0.0016899371254598748 False
duiker 0.0009156356669827548 False
eland 0.001496580786512857 False
elephant 0.002729778089449364 True
empty 0.028853605698679476 True
gazellegrants 0.0065368292214597395 False
gazellethomsons 0.011717870501081357 True
genet 0.00032101311571715324 False
giraffe 0.0018571718499794357 False
guineafowl 0.0010298060151850323 False
hare 0.0004212891333290041 False
hartebeest 0.0028648895539727377 False
hippopotamus 0.0006920607192706828 False
honeybadger 9.990881401212624e-05 False
hyenaspotted 0.0028974383025049876 False
hyenastriped 0.0002566949206167334 False
impala 0.003891201978652512 True
insectspider 0.002

In [70]:
test_dataset.pred_model_480_360_as480_img3_empty.min()

0.0

In [71]:
columns2 = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] + \
['pred_back_as480_' + c for c in feature_classes] +\
['pred_model_480_360_as480_img2_' + c for c in feature_classes] + \
['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] + \
['pred_model_480_360_as480_img3_' + c for c in feature_classes] + \
['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] + \
['count_imgs', 'hour', 'month']


X_test = test_dataset[columns2]

# X_test.loc[X_test.count_imgs<3, ['pred_model_480_360_as480_img3_' + c for c in train_labels.columns]]=-0.01
# X_test.loc[X_test.count_imgs<2, ['pred_model_480_360_as480_img2_' + c for c in train_labels.columns]]=-0.01
# X_test.loc[X_test.count_imgs<3, ['pred_model_480_360_v2_as480_img3_' + c for c in train_labels.columns]]=-0.01
# X_test.loc[X_test.count_imgs<2, ['pred_model_480_360_v2_as480_img2_' + c for c in train_labels.columns]]=-0.01
# X_test.loc[X_test.count_imgs<2, ['pred_back_as480_' + c for c in train_labels.columns]]=-0.01

ls = 0
c = list(train_labels.columns)
c.remove('reptiles')
import os
for clazz in c:
    is_model = False

    y_test = test_dataset[clazz]
    if sum(y_test) ==0 :
        print(clazz, 'bad')
        continue
    if not os.path.isfile("lgbs_v4/" + clazz + '.txt'):
        x = test_dataset['mean_with_combines_imgs_' + clazz].values
    else:
        try:
            model = lgb.Booster(model_file="lgbs_v4/" + clazz + '.txt')
            x=model.predict(X_test)
            is_model=True
        except:
            x = test_dataset['mean_with_combines_imgs_' + clazz].values

    
    l =log_loss(y_test,x )
    ls+=l
    print(clazz, l, is_model)
print(ls/54)

aardvark 0.00012400494331310212 False
aardwolf 0.00041863614940684037 False
baboon 0.0014443456538042966 True
bat bad
batearedfox 0.0005680056603519342 False
buffalo 0.0041381664740637895 True
bushbuck 0.00019572546624137754 False
caracal 0.0001380608397927396 False
cattle bad
cheetah 0.0013401865619093502 False
civet 2.990007267168342e-05 False
dikdik 0.0016899371254598748 False
duiker 0.0009156356669827548 False
eland 0.001496580786512857 False
elephant 0.003379812923626614 True
empty 0.03926864654872551 True
gazellegrants 0.006501083926210223 True
gazellethomsons 0.011885328002159195 True
genet 0.00032101311571715324 False
giraffe 0.0014248888024763267 True
guineafowl 0.0011434726643640663 True
hare 0.0006365879608942139 True
hartebeest 0.001914180663235989 True
hippopotamus 0.0006920607192706828 False
honeybadger 2.7585898667224167e-05 True
hyenaspotted 0.0027635968508749782 True
hyenastriped 0.0002566949206167334 False
impala 0.004071047884089401 True
insectspider 0.00284062727273

In [ ]:

0.0034590009769709116


In [61]:
list(train_labels.columns)

['aardvark',
 'aardwolf',
 'baboon',
 'bat',
 'batearedfox',
 'buffalo',
 'bushbuck',
 'caracal',
 'cattle',
 'cheetah',
 'civet',
 'dikdik',
 'duiker',
 'eland',
 'elephant',
 'empty',
 'gazellegrants',
 'gazellethomsons',
 'genet',
 'giraffe',
 'guineafowl',
 'hare',
 'hartebeest',
 'hippopotamus',
 'honeybadger',
 'hyenaspotted',
 'hyenastriped',
 'impala',
 'insectspider',
 'jackal',
 'koribustard',
 'leopard',
 'lionfemale',
 'lionmale',
 'mongoose',
 'monkeyvervet',
 'ostrich',
 'otherbird',
 'porcupine',
 'reedbuck',
 'reptiles',
 'rhinoceros',
 'rodents',
 'secretarybird',
 'serval',
 'steenbok',
 'topi',
 'vulture',
 'warthog',
 'waterbuck',
 'wildcat',
 'wildebeest',
 'zebra',
 'zorilla']